# Análisis Exploratorio del dataset de Ventas de Lácteos 2024
Este notebook carga el CSV procesado para observar dimensiones, tipos, nulos y duplicados; se guardan luego gráficas y tablas en `src/results` para documentar Hallazgos y preparar el análisis inferencial.

## Configuración
Utilizamos `Path` para mantener rutas relativas y `pandas`, `seaborn` con tipado ligero para trabajar con los datos.

In [6]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme(style='whitegrid')
RESULTS_DIR = Path('src') / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

def cargar_dataset_procesado(ruta: Path) -> pd.DataFrame:
    if not ruta.exists():
        raise FileNotFoundError(f'Ejecuta el pipeline de transformación antes de correr este notebook: {ruta}')
    return pd.read_csv(ruta)

RUTA_PROCESADA = Path('data') / 'processed' / 'ventas_lacteos_2024.csv'
df_procesado = cargar_dataset_procesado(RUTA_PROCESADA)
df_procesado


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

## Inspección básica
Revisamos dimensiones, tipos, nulos y duplicados para reconocer calidad de datos antes de graficar.

In [ ]:
print('Dimensiones del dataset:', df_procesado.shape)
print('Tipos de datos:
', df_procesado.dtypes)
print('Valores nulos por columna:
', df_procesado.isnull().sum())
print('Registros duplicados:', df_procesado.duplicated().sum())
df_procesado.head()

## Estadísticas por categoría
Agrupamos `valor_total_usd` por `categoria` para guardar una tabla que luego incorporaremos en el README y SPSS.

In [ ]:
tabla_categorias = (
    df_procesado
    .groupby('categoria', sort=True)
    .agg(
        registros=('valor_total_usd', 'count'),
        media=('valor_total_usd', 'mean'),
        desviacion=('valor_total_usd', 'std'),
        minimo=('valor_total_usd', 'min'),
        maximo=('valor_total_usd', 'max'),
    )
    .sort_values('registros', ascending=False)
)
tabla_categorias.to_html(RESULTS_DIR / 'tabla_estadisticas_categorias.html', index=True, float_format='%.2f')
tabla_categorias

## Visualizaciones clave
Generamos histogramas y gráficos por categoría. Cada figura se guarda en `src/results` para reproducibilidad.

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df_procesado['valor_total_usd'], bins=40, kde=True, color='#1f77b4')
plt.title('Distribución del valor total (USD) por transacción')
plt.xlabel('Valor total (USD)')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'hist_valor_total.jpg')
plt.show()

In [ ]:
plt.figure(figsize=(9, 5))
tabla_categorias_plot = tabla_categorias.reset_index()
sns.barplot(
    data=tabla_categorias_plot,
    x='registros',
    y='categoria',
    palette='viridis'
)
plt.title('Transacciones por categoría')
plt.xlabel('Número de transacciones')
plt.ylabel('Categoría')
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'barras_categoria.jpg')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(
    data=df_procesado,
    x='categoria',
    y='valor_total_usd',
    palette='rocket',
    showfliers=False
)
plt.title('Rango del valor total por categoría')
plt.xlabel('Categoría')
plt.ylabel('Valor total (USD)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'boxplot_categoria.jpg')
plt.show()

## Hallazgos
1. Las categorías clave (p. ej., Quesos, Leche, Yogur) dominan tanto el número de transacciones como el valor promedio, lo cual sugiere priorizar su análisis inferencial.
2. La distribución del valor total presenta una cola derecha moderada, así que se debe vigilar la homogeneidad de varianzas en la prueba.
3. Las gráficas y la tabla quedan almacenadas en `src/results`, por lo que las podemos usar en el informe y en SPSS.